In [1]:
from bs4 import BeautifulSoup as BS
import urllib as url
import pandas as pd
import numpy as np  
import re
from datetime import datetime


In [2]:
request = url.request.Request("https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900")
result = url.request.urlopen(request)
resulttext = result.read()

In [3]:
soup = BS(resulttext, 'html.parser')


In [4]:
# the html has a number of tables and we want to  have a table that we are looking for , so that we explictly 
#pass the characterstics of the tables  through class_ as an argument . We find the characterstics from the html 

earthquake_table_list = soup.find_all('table', class_=['sortable', 'wikitable', 'jquery-tablesorter'])

# we will assert if we have got the table that we want for , we wnat only one table

assert len(earthquake_table_list) == 1

#  since we have the list , we  want to convert the list in to table (have to check this again)

earthquake_table = earthquake_table_list[0]

In [5]:
# once we have the table, we want to extract the table header represented as 'th' by using the find_all, find_all 

table_head = earthquake_table.find_all('th')

# here we are assigning the table for  header by looping through table head . 
#The result of the .text() method is a string containing the combined text of all matched elements.

columns = [th.text for th in table_head]

In [6]:
print(columns)

['Origin (UTC)', 'Present-day country and link to Wikipedia article', 'Lat', 'Long', 'Depth (km)', 'Magnitude', 'Secondary Effects', 'PDE Shaking Deaths', 'PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', 'Other Source Deaths']


In [7]:
# find_all the rows of the table and skip the first column as the table header is one child it self.

table_rows = earthquake_table.find_all('tr')[1:]

#do a loop , create an empty set for  and store it as df_rows

df_rows = list()
for row in table_rows:
    row_data= [data.text for data in row.find_all('td')]
    
    df_rows.append(row_data)

In [8]:
for row in df_rows[:5]:
    print(row)

['1900-05-11 17:23', 'Japan', '38.700', '141.100', '5', '7.0 MJMA', '', '', '', '', '', '']
['1900-07-12 06:25', 'Turkey', '40.300', '43.100', '', '5.9 Muk', '', '', '', '140', '', '']
['1900-10-29 09:11', 'Venezuela', '11.000', '-66.000', '0', '7.7 Mw', '', '', '', '', '', '']
['1901-02-15 00:00', 'China', '26.000', '100.100', '0', '6.5 Ms', '', '', '', '', '', '']
['1901-03-31 07:11', 'Bulgaria', '43.400', '28.700', '', '6.4 Muk', '', '', '', '4', '', '']


In [9]:
#   the next step is creating a data frame , where we combine the two tables  row and column tables.# the data and the header 
earthquake_df = pd.DataFrame(data=df_rows, columns=columns)
#earthquake_df.loc[earthquake_df['Present-day country and link to Wikipedia article']=='Iran']



In [10]:
#earthquake_df.replace(r'^\s*$', np.nan, inplace = True)

In [11]:
earthquake_df=earthquake_df.replace('', np.NaN)

In [12]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 12 columns):
Origin (UTC)                                         1340 non-null object
Present-day country and link to Wikipedia article    1340 non-null object
Lat                                                  1326 non-null object
Long                                                 1326 non-null object
Depth (km)                                           1250 non-null object
Magnitude                                            1339 non-null object
Secondary Effects                                    373 non-null object
PDE Shaking Deaths                                   739 non-null object
PDE Total Deaths                                     750 non-null object
Utsu Total Deaths                                    1027 non-null object
EM-DAT Total Deaths                                  560 non-null object
Other Source Deaths                                  37 non-null object
dtypes: obj

In [13]:
# renaming a columns, is  there any other mechanism to rename a target column alone
earthquake_df.columns=["Origin(UTC)","country","Lat","Long", "Depth(km)","Magnitude","Secondary Effects","PDE Shaking Deaths",
                       "PDE Total Deaths","Utsu Total Deaths","EM-DAT Total Deaths","Other Source Deaths"]

In [14]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 12 columns):
Origin(UTC)            1340 non-null object
country                1340 non-null object
Lat                    1326 non-null object
Long                   1326 non-null object
Depth(km)              1250 non-null object
Magnitude              1339 non-null object
Secondary Effects      373 non-null object
PDE Shaking Deaths     739 non-null object
PDE Total Deaths       750 non-null object
Utsu Total Deaths      1027 non-null object
EM-DAT Total Deaths    560 non-null object
Other Source Deaths    37 non-null object
dtypes: object(12)
memory usage: 125.7+ KB


In [15]:
# fixing issues with country columns  to have the unique name for each country.
# we need to make a decesion on the SaudiArabia/Egybt  observation

test="\s\(.*"
earthquake_df['country']= earthquake_df['country'].str.replace(test, "")
test2= "\,.*"
earthquake_df['country']=earthquake_df['country'].str.replace(test2, "")
test3="United States Minor.*"
earthquake_df['country']=earthquake_df['country'].str.replace(test3,"United States")
test4="US T.*"
earthquake_df['country']=earthquake_df['country'].str.replace(test4,"United States")
test5="\(.*"
earthquake_df['country']=earthquake_df['country'].str.replace(test5,"Venezuela")

In [16]:
earthquake_df['country'].value_counts()
# here we are observing  that a country name followed by ',' [Iran, 2005 Qeshm earthquake ]                              1
#and  country names with parenthesis . 

China                                  166
Indonesia                               96
Turkey                                  95
Iran                                    92
Japan                                   82
Peru                                    54
Taiwan                                  48
Mexico                                  46
Greece                                  44
Chile                                   44
Philippines                             39
United States                           37
Italy                                   37
Afghanistan                             30
Pakistan                                26
India                                   25
Colombia                                24
Algeria                                 21
Ecuador                                 19
Papua New Guinea                        17
Guatemala                               15
South Africa                            14
Russian Federation                      13
Venezuela  

In [17]:
#splitted_country=earthquake_df['country'].str.split('(')
#splitted_country
#type(splitted_country)
#earthquake_df['country']=splitted_country
#earthquake_df

In [18]:
#for date_str in earthquake_df["Origin(UTC)]":
   # Origin(UTC) = datetime.strptime(date_str,'%m/%d/%Y')
   # print(Origin(UTC))

In [19]:
#Data cleaning tasks include:

#Replace empty strings with NaN
#Remove the footnotes from the 'Other Source Deaths' column
#Convert Magnitude to a numeric
#Create a new column ('deaths') that evaluates the four total-death columns ('PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', and 'Other Source Deaths') and populates the new column with the highest value.
#Explore the data in terms of when and where earthquakes occurred and how severe they were (magnitude, deaths, secondary effects).

In [20]:
earthquake_df['Other Source Deaths'].value_counts()

1                                      3
79                                     1
295                                    1
41                                     1
181                                    1
2489[10]                               1
380                                    1
15894[20]                              1
1500[6]                                1
2041                                   1
231000*[13] 283000*[14] 227898*[15]    1
60[16]                                 1
45000[9]                               1
68                                     1
521[19]                                1
189                                    1
601 (as of October 30, 2011)           1
215                                    1
34                                     1
33[8]                                  1
164[6]                                 1
46                                     1
8000+                                  1
120                                    1
2,698           

In [21]:
#earthquake_df['Other Source Deaths']
earthquake_df[earthquake_df['Other Source Deaths'].str.contains('26000', na=False)]

,Origin(UTC),country,Lat,Long,Depth(km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths
1206,2003-12-26 01:56,Iran,28.950,58.268,6,6.6 Mw,L,31000,31000,NaN,26796,26271[11] 26000[12]


In [22]:
# removing the footnotes and commas between numbers in [Other Source Deaths]
test6="\[\d+]|\+|\s\(.*|\*"
earthquake_df['Other Source Deaths']=earthquake_df['Other Source Deaths'].str.replace(test6,"")
test7="\,"
earthquake_df['Other Source Deaths']=earthquake_df['Other Source Deaths'].str.replace(test7,"")

In [23]:
earthquake_df['Other Source Deaths'].value_counts()

1                       3
79                      1
164                     1
215                     1
41                      1
181                     1
1500                    1
380                     1
150                     1
2041                    1
2489                    1
68                      1
231000 283000 227898    1
2698                    1
189                     1
60                      1
222517                  1
33                      1
45000                   1
295                     1
601                     1
34                      1
1115                    1
46                      1
120                     1
1404                    1
111                     1
8000                    1
26271 26000             1
521                     1
200                     1
15894                   1
105000                  1
42                      1
3500                    1
Name: Other Source Deaths, dtype: int64

In [24]:
means = []
for row in earthquake_df["Other Source Deaths"]:
    # print("\noriginal: {}".format(type(row)))
    #if isinstance(row, float):
    #    int_list = [str(row)]
    if isinstance(row, str):
        #print(len(row))
        int_list = row.strip().split(" ")
        print("transformed: {}".format(int_list))
        #row.split(" ")
        #make_int = list(map(int, int_list))
        #print("new type:", make_int)
        total = sum(list(map(int, int_list)))
        number = len(int_list)
        mean = total/number
        means.append(mean)
        print(mean)
    else:
        means.append(row)
        
earthquake_df["Other Source Deaths"] = means
    
    #int_list = row.str.split(" ")
    #print(int_list)

transformed: ['3500']
3500.0
transformed: ['8000']
8000.0
transformed: ['46']
46.0
transformed: ['164']
164.0
transformed: ['1500']
1500.0
transformed: ['105000']
105000.0
transformed: ['1404']
1404.0
transformed: ['1']
1.0
transformed: ['120']
120.0
transformed: ['200']
200.0
transformed: ['380']
380.0
transformed: ['2041']
2041.0
transformed: ['33']
33.0
transformed: ['45000']
45000.0
transformed: ['2489']
2489.0
transformed: ['26271', '26000']
26135.5
transformed: ['68']
68.0
transformed: ['231000', '283000', '227898']
247299.33333333334
transformed: ['41']
41.0
transformed: ['60']
60.0
transformed: ['215']
215.0
transformed: ['34']
34.0
transformed: ['295']
295.0
transformed: ['1']
1.0
transformed: ['79']
79.0
transformed: ['189']
189.0
transformed: ['1115']
1115.0
transformed: ['222517']
222517.0
transformed: ['521']
521.0
transformed: ['42']
42.0
transformed: ['1']
1.0
transformed: ['2698']
2698.0
transformed: ['181']
181.0
transformed: ['15894']
15894.0
transformed: ['150']
150.

In [25]:
d_types = set([type(val) for val in earthquake_df['Magnitude']])

In [26]:
d_types

{float, str}

In [27]:
# Converting magnitude to numeric 

earthquake_df['magnitude']=earthquake_df['Magnitude'].str.split(' ').str.get(0)
earthquake_df['magnitude']

d_types= set([type(val) for val in earthquake_df['magnitude']])
d_types

earthquake_df['magnitude'].dtype

earthquake_df.magnitude.values

earthquake_df.magnitude=earthquake_df.magnitude.astype(float).fillna(0.0)
earthquake_df.magnitude.values
earthquake_df['magnitude'].dtype

dtype('float64')

In [28]:
y=earthquake_df['Magnitude'].str.split(' ').str.get(1)
y.unique()

array(['MJMA', 'Muk', 'Mw', 'Ms', 'ML', 'mb', nan, 'Lg', 'Md', 'Mb'], dtype=object)

In [29]:
#Based on what literature has provided to us on the possible conversion between different units,
#I am  planning to convert units in to a standard or common units. I have found a literature that has 
# a subtle information on the difference between each  units, I think it is beyond my capacity to handle
#https://www.researchgate.net/profile/Emmanuel_Scordilis/publication/292771405_Globally_valid_relations_converting_Ms_mb_and_MJMA_to_Mw/
#links/56b8352e08ae5ad3605ddba1/Globally-valid-relations-converting-Ms-mb-and-MJMA-to-Mw.pdf


#def (x):
  #  y=earthquake_df['Magnitude'].str.split('')
    
  #  if y[1]=='MJMA':
   #     return float(y[0])
    #elif y[1]=='Muk':
     #   return float(y[0])
   # elif y[1]=='Mw':
   #     return float(y[0])
   # elif y[1]=='Ms':
    #    return float(y[0])
   # elif y[1]=='ML':
     #   return float(y[0])
  #  elif y[1]=='mb':
     #   return float(y[0])
   # elif y[1]=='Lg':
     #   return float(y[0])
   # elif y[1]=='Md':
    #    return float(y[0])
   # elif y[1]=='MB':
   #     return float(y[0])
   # else:
   #     return NaN
#earthquake_df['']

#earthquake_df['Magnitude_c']=earthquake_df.Magnitude.apply(lambda x: y(x))

In [30]:
#Create a new column ('deaths')  FROM TOTAL DEATH COLUMNS AND POPULATES WITH THE HIGHEST VALUE


In [31]:
#Explore the data in terms of when and where earthquakes occurred and how severe they were (magnitude, deaths, secondary effects).
earthquake_df['Other Source Deaths']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16        3500.0
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22           NaN
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
          ...   
1310         NaN
1311         NaN
1312         NaN
1313         NaN
1314         NaN
1315         NaN
1316         NaN
1317        41.0
1318        60.0
1319         NaN
1320       215.0
1321        34.0
1322       295.0
1323         NaN
1324         1.0
1325        79.0
1326       189.0
1327      1115.0
1328    222517.0
1329       521.0
1330        42.0
1331         1.0
1332      2698.0
1333       181.0
1334         NaN
1335     15894.0
1336       150.0
1337         N

In [32]:
x=earthquake_df.loc[earthquake_df['country']=='Iran']
x['Other Source Deaths'].value_counts()

26135.5    1
8000.0     1
Name: Other Source Deaths, dtype: int64

In [33]:
#To select rows whose column value equals a scalar, some_value, use ==:

#df.loc[df['column_name'] == some_value]
#To select rows whose column value is in an iterable, some_values, use isin:

#df.loc[df['column_name'].isin(some_values)]
#Combine multiple conditions with &:

#df.loc[(df['column_name'] == some_value) & df['other_column'].isin(some_values)]
#To select rows whose column value does not equal some_value, use !=:

#df.loc[df['column_name'] != some_value]
#isin returns a boolean Series, so to select rows whose value is not in some_values, negate the boolean Series using ~:

#df.loc[~df['column_name'].isin(some_values)]

In [34]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 13 columns):
Origin(UTC)            1340 non-null object
country                1340 non-null object
Lat                    1326 non-null object
Long                   1326 non-null object
Depth(km)              1250 non-null object
Magnitude              1339 non-null object
Secondary Effects      373 non-null object
PDE Shaking Deaths     739 non-null object
PDE Total Deaths       750 non-null object
Utsu Total Deaths      1027 non-null object
EM-DAT Total Deaths    560 non-null object
Other Source Deaths    37 non-null float64
magnitude              1340 non-null float64
dtypes: float64(2), object(11)
memory usage: 136.2+ KB


In [58]:
float(earthquake_df['PDE Total Deaths'])
  #  ', 'Utsu Total Deaths','EM-DAT Total Deaths', 'Other Source Deaths'])

TypeError: cannot convert the series to <class 'float'>

In [36]:
type(earthquake_df['PDE Total Deaths'])

pandas.core.series.Series

In [37]:
d_types = set([type(val) for val in earthquake_df['Utsu Total Deaths']])
d_types

{float, str}

In [38]:
percapita=pd.read_csv("UN_GDP.csv")

In [39]:
percapita.head()

,Country or Area,Year,Item,Value
0,Afghanistan,2015,Gross Domestic Product (GDP),623.184798
1,Afghanistan,2014,Gross Domestic Product (GDP),667.883420
2,Afghanistan,2013,Gross Domestic Product (GDP),704.322178
3,Afghanistan,2012,Gross Domestic Product (GDP),717.563696
4,Afghanistan,2011,Gross Domestic Product (GDP),665.429433


In [40]:
pop=pd.read_csv("UN_total_population.csv")

In [41]:
pop.head()

,Country or Area,Subgroup,Year,Source,Unit,Value,Value Footnotes
0,Afghanistan,Total,2011.0,UN_United Nations Population Division,Thousand,32358.0,NaN
1,Albania,Total,2011.0,UN_United Nations Population Division,Thousand,3216.0,NaN
2,Algeria,Total,2011.0,UN_United Nations Population Division,Thousand,35980.0,NaN
3,Andorra,Total,2011.0,UN_United Nations Population Division,Thousand,86.0,NaN
4,Angola,Total,2011.0,UN_United Nations Population Division,Thousand,19618.0,NaN


In [42]:
type(pop)

pandas.core.frame.DataFrame

In [43]:
type(pop.index)

pandas.core.indexes.range.RangeIndex

In [47]:
float(earthquake_df['Total Deaths']=earthquake_df[['PDE Total Deaths', 'Utsu Total Deaths','EM-DAT Total Deaths', 'Other Source Deaths']].max(axis=1)

In [53]:
earthquake_df['Total Deaths'].head(20)

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16    3500.0
17       NaN
18       NaN
19       NaN
Name: Total Deaths, dtype: float64

In [67]:
cols=['PDE Total Deaths', 'Utsu Total Deaths','EM-DAT Total Deaths', 'Other Source Deaths', 'Total Deaths']
earthquake_df.cols=earthquake_df.cols.apply(pd.to_numeric, error='coherce')
earthquake_df[['PDE Total Deaths', 'Utsu Total Deaths','EM-DAT Total Deaths', 'Other Source Deaths', 'Total Deaths']].head()

AttributeError: 'DataFrame' object has no attribute 'cols'

In [72]:
# earthquake_df.apply(pd.to_numeric, errors='ignore').info()
cols=['PDE Total Deaths', 'Utsu Total Deaths','EM-DAT Total Deaths', 'Other Source Deaths', 'Total Deaths']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 14 columns):
Origin(UTC)            1340 non-null object
country                1340 non-null object
Lat                    1326 non-null object
Long                   1326 non-null object
Depth(km)              1250 non-null object
Magnitude              1339 non-null object
Secondary Effects      373 non-null object
PDE Shaking Deaths     739 non-null float64
PDE Total Deaths       750 non-null float64
Utsu Total Deaths      1027 non-null float64
EM-DAT Total Deaths    560 non-null object
Other Source Deaths    37 non-null float64
magnitude              1340 non-null float64
Total Deaths           37 non-null float64
dtypes: float64(6), object(8)
memory usage: 146.6+ KB
